In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Boolean, DateTime, JSON
import uuid
import datetime
import requests
import json


# Create engine
engine = create_engine('postgresql://0.0.0.0/postgres?user=postgres&password=mysecretpassword')

# Create session
Session = sessionmaker(bind=engine)
session = Session()

# Create Base
Base = declarative_base()

# Define User table
class User(Base):
    __tablename__ = 'user'

    id = Column(String(36), primary_key=True, default=lambda: str(uuid.uuid4()), unique=True)
    email = Column(String(120), unique=True, nullable=False)
    first_name = Column(String(50), nullable=False)
    last_name = Column(String(50), nullable=False)
    phone_number = Column(String(20), nullable=True)
    provider_token = Column(JSON, nullable=True)
    password_hash = Column(String(256), nullable=False)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.datetime.utcnow, onupdate=datetime.datetime.utcnow)
    provider = Column(String(20), nullable=False, default='local') # 'local', 'gmail', 'dropbox'
    provider_id = Column(String(120), nullable=True) # id provided by 'gmail' or 'dropbox'


# Fetch data from User table
users = session.query(User).all()

def login_user(email, password):
    url = "http://localhost:80/login"
    data = {
        "email": email,
        "password": password
    }
    response = requests.post(url, json=data)
    
    return response.json()



/var/folders/1f/f_5tqnhs7bzf0gkc3zspcyf80000gn/T/ipykernel_53923/3743867923.py:19: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [8]:
user_this = login_user("newemail@example.com", "password")
user_this

{'token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJmcmVzaCI6ZmFsc2UsImlhdCI6MTcxMzM0ODg1NiwianRpIjoiZmZlMTVjY2QtZGIxMS00NWQ4LWE3ODUtNThjYjdmMjk5YTVjIiwidHlwZSI6ImFjY2VzcyIsInN1YiI6eyJpZCI6IjQ5Mzk0YmFmLWVjZjAtNDUyNy1iYmE5LWRlNTI2NzczYWU5ZSIsImVtYWlsIjoibmV3ZW1haWxAZXhhbXBsZS5jb20iLCJmaXJzdF9uYW1lIjoiTmV3Rmlyc3ROYW1lIiwibGFzdF9uYW1lIjoiTmV3TGFzdE5hbWUiLCJwaG9uZV9udW1iZXIiOiIxMjM0NTY3ODkwIiwiaXNfYWN0aXZlIjp0cnVlLCJjcmVhdGVkX2F0IjoiV2VkLCAxNyBBcHIgMjAyNCAwOTozNzoxNCBHTVQiLCJ1cGRhdGVkX2F0IjoiV2VkLCAxNyBBcHIgMjAyNCAwOTo1NzozOSBHTVQiLCJwcm92aWRlciI6ImxvY2FsIiwicHJvdmlkZXJfaWQiOm51bGx9LCJuYmYiOjE3MTMzNDg4NTYsImNzcmYiOiI5YmEzOTZhZi0zNjJkLTQ0YTEtOTk1NS05MGY0OThjYmVjZDYiLCJleHAiOjE3MTMzNDk3NTZ9.XMvxeT-EGDLnHifTiwMdpFXrV50QtAWG-pWWdvE7Ph0',
 'user_id': '49394baf-ecf0-4527-bba9-de526773ae9e'}

In [9]:
# Fetch provider_token from User table for the given user_id
provider_token = session.query(User.provider_token).filter(User.id == user_this["user_id"]).first()

In [16]:
provider_token = json.loads(provider_token[0])

### DROP-BOX

In [30]:
#HELPER
from dropbox import DropboxOAuth2Flow, Dropbox, files
def list_files_and_folders(folder_path, access_token, file_types=None):
    if not access_token:
        return {'error': 'Dropbox authentication required'}

    dbx = Dropbox(access_token)
    try:
        files_and_folders = []
        for entry in dbx.files_list_folder(folder_path).entries:
            if isinstance(entry, files.FolderMetadata):
                files_and_folders.append({
                    'name': entry.name, 
                    'is_directory': True, 
                    'id': entry.id, 
                    'path_display': entry.path_display, 
                    'path_lower': entry.path_lower
                })
            elif isinstance(entry, files.FileMetadata):
                if not file_types or entry.name.endswith(tuple(file_types)):
                    files_and_folders.append({
                        'name': entry.name, 
                        'is_directory': False, 
                        'client_modified': entry.client_modified, 
                        'server_modified': entry.server_modified, 
                        'rev': entry.rev, 
                        'size': entry.size, 
                        'id': entry.id, 
                        'path_display': entry.path_display, 
                        'path_lower': entry.path_lower
                    })
        return {'files_and_folders': files_and_folders}
    except Exception as e:
        return{'error': str(e)}
    
    

# Function to download a file from Dropbox using its id
def download_file(file_id, access_token):
    if not access_token:
        return {'error': 'Dropbox authentication required'}

    dbx = Dropbox(access_token)
    try:
        metadata, res = dbx.files_download(file_id)
        file_data = res.content
        return {'file_data': file_data, 'metadata': metadata}
    except Exception as e:
        return {'error': str(e)}

# Function to get files and folders recursively from a given folder path
def list_files_and_folders_recursively(folder_path, access_token, file_types=None):
    if not access_token:
        return {'error': 'Dropbox authentication required'}

    dbx = Dropbox(access_token)
    try:
        files_and_folders = []
        for entry in dbx.files_list_folder(folder_path, recursive=True).entries:
            if isinstance(entry, files.FolderMetadata):
                files_and_folders.append({
                    'name': entry.name, 
                    'is_directory': True, 
                    'id': entry.id, 
                    'path_display': entry.path_display, 
                    'path_lower': entry.path_lower
                })
            elif isinstance(entry, files.FileMetadata):
                if not file_types or entry.name.endswith(tuple(file_types)):
                    files_and_folders.append({
                        'name': entry.name, 
                        'is_directory': False, 
                        'client_modified': entry.client_modified, 
                        'server_modified': entry.server_modified, 
                        'rev': entry.rev, 
                        'size': entry.size, 
                        'id': entry.id, 
                        'path_display': entry.path_display, 
                        'path_lower': entry.path_lower
                    })
        return {'files_and_folders': files_and_folders}
    except Exception as e:
        return {'error': str(e)}
    




In [31]:
ext = ["png"]
path = '' #"/ace"
list_files_and_folders(path,provider_token["access_token"], ext)['files_and_folders']

[{'name': 'ace',
  'is_directory': True,
  'id': 'id:bPLK2gOQtIcAAAAAAAAAEQ',
  'path_display': '/ace',
  'path_lower': '/ace'},
 {'name': 'Screenshot 2024-04-16 at 11.32.10\u202fPM.png',
  'is_directory': False,
  'client_modified': datetime.datetime(2024, 4, 17, 9, 51, 28),
  'server_modified': datetime.datetime(2024, 4, 17, 9, 51, 28),
  'rev': '61647cc71197794d1fa93',
  'size': 463658,
  'id': 'id:bPLK2gOQtIcAAAAAAAAADg',
  'path_display': '/Screenshot 2024-04-16 at 11.32.10\u202fPM.png',
  'path_lower': '/screenshot 2024-04-16 at 11.32.10\u202fpm.png'},
 {'name': 'Screenshot 2024-04-16 at 11.32.39\u202fPM.png',
  'is_directory': False,
  'client_modified': datetime.datetime(2024, 4, 17, 9, 51, 28),
  'server_modified': datetime.datetime(2024, 4, 17, 9, 51, 28),
  'rev': '61647cc71197894d1fa93',
  'size': 512859,
  'id': 'id:bPLK2gOQtIcAAAAAAAAADw',
  'path_display': '/Screenshot 2024-04-16 at 11.32.39\u202fPM.png',
  'path_lower': '/screenshot 2024-04-16 at 11.32.39\u202fpm.png'}]

In [32]:
list_files_and_folders_recursively(path,provider_token["access_token"])['files_and_folders']

[{'name': 'ace',
  'is_directory': True,
  'id': 'id:bPLK2gOQtIcAAAAAAAAAEQ',
  'path_display': '/ace',
  'path_lower': '/ace'},
 {'name': 'user.json',
  'is_directory': False,
  'client_modified': datetime.datetime(2024, 4, 17, 9, 51, 27),
  'server_modified': datetime.datetime(2024, 4, 17, 9, 51, 27),
  'rev': '61647cc63a09894d1fa93',
  'size': 56,
  'id': 'id:bPLK2gOQtIcAAAAAAAAADA',
  'path_display': '/user.json',
  'path_lower': '/user.json'},
 {'name': 'queue.json',
  'is_directory': False,
  'client_modified': datetime.datetime(2024, 4, 17, 9, 51, 28),
  'server_modified': datetime.datetime(2024, 4, 17, 9, 51, 28),
  'rev': '61647cc71197694d1fa93',
  'size': 671,
  'id': 'id:bPLK2gOQtIcAAAAAAAAADQ',
  'path_display': '/queue.json',
  'path_lower': '/queue.json'},
 {'name': 'Screenshot 2024-04-16 at 11.32.10\u202fPM.png',
  'is_directory': False,
  'client_modified': datetime.datetime(2024, 4, 17, 9, 51, 28),
  'server_modified': datetime.datetime(2024, 4, 17, 9, 51, 28),
  'rev'

In [46]:
id= "id:bPLK2gOQtIcAAAAAAAAAFQ"
this_file = download_file(id, provider_token["access_token"])

In [48]:
this_file

{'file_data': b'PK\x03\x04\x14\x00\x08\x08\x08\x00\x1c@JW\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x12\x00\x00\x00word/numbering.xml\xed\x9dYn\xe3F\x10\x86\xdfs\n\x81@\x1em\xee\x8b\x84\x91\x07\xb4\x96\xc0A\x10\x04\x18\xe7\x00-\xaa%\x11&\x9b\x04I\xc9\xe3<\xe6R9V\xae\x90\xe6&Y3fG!\xa5h\xc1\xef\x17\x9a\xac\xae\xea\xaa\x9fT\xb3>\xb4\r}\xfa\xfc5\x0cz\x1b\x9a\xa4~\xc4\x86\x92z\xafH=\xca\xbch\xee\xb3\xe5P\xfa\xfdyz\xe7H\xbd4#lN\x82\x88\xd1\xa1\xf4FS\xe9\xf3\xc3\x0f\x9f^\x07l\x1d\xceh\xc2\xc7\xf5x\x08\x96\x0e^co(\xad\xb2,\x1e\xc8r\xea\xadhH\xd2\xfb\xd0\xf7\x92(\x8d\x16\xd9\xbd\x17\x85r\xb4X\xf8\x1e\x95_\xa3d.k\x8a\xaa\x14\xbf\xc5I\xe4\xd14\xe5qF\x84mH*U\xe1\xc2\xef\xa3E1e\xdc\xb8\x88\x92\x90d\xfc4Y\xca!I^\xd6\xf1\x1d\x8f\x1e\x93\xcc\x9f\xf9\x81\x9f\xbd\xf1\xd8\x8aU\x87\x89\x86\xd2:a\x83*\xc4\xdd6\xa1\xdceP&T\x1dj\x8f\xe4\x90yc\xe2\xbd\x90%-\xa6\x92\x13\x1a\xf0\xc9#\x96\xae\xfcx\x97\xff!a\xca\x99\xc7\x91\xb7\x0e)\xcb\xcah\xdc\xb8\xaa\x83lD\xd9o\xc2@\xdaj\xaf\x1a\xdd\xc4\x1f\'\xe4\x95\x1f

In [51]:
content = this_file['file_data']
ext = this_file["metadata"].path_display.split(".")[-1]
with open(f"output.{ext}", "wb") as f:
    f.write(content)
